# Item37: Compose Classes Instead of Nesting Many Levels of Built-in Types

## 首先看一个比较简单的例子

- 我们的目标是制作一个`SimpleGradebook`: 
  - 添加学生
  - 添加成绩
  - 求某一个学生的成绩的平均值

In [1]:
class SimpleGradebook:

    def __init__(self) -> None:
        self._grades = {}

    def add_student(self, name):
        self._grades[name] = []


    def report_grade(self, name, score):
        self._grades[name].append(score)

    def average_grade(self, name):
        grades = self._grades[name]

        return sum(grades)/len(grades)


In [2]:
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
book.report_grade('Isaac Newton', 95)
book.report_grade('Isaac Newton', 85)

book.average_grade('Isaac Newton')

90.0

## 需要添加科目的时候

- 此时需要`defaultdict`
  - 实际上是一个nesting的制作工厂, 之后的使用也是一个字典, 但是限定了value的类型, 此处有一个好处, 例如此处使用list, 他直接初始化后会有一个list的方法: `name: value: list`, 使用如下面的应用所示, 可以想想如果不使用该方法的话下面的内容要如何实现.

In [3]:
from collections import defaultdict

class BySubjectGradebook:

    def __init__(self):
        self._grades = {}

    def add_students(self, name):
        self._grades[name] = defaultdict(list)

    def report_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0

        for grades in by_subject.values():
            total+=sum(grades)
            count+=len(grades)
        
        return total/count


- 但是如果说我此时还要再往里面加东西的话, 我又要重新写代码了, 比如说我的每一个score都是有权重的

In [4]:
grades = []
grades.append([85, 0.6])
grades.append([95, 0.19])

total = sum(grade*weight for grade, weight in grades) 
weights = sum(weight for _, weight in grades)

total / weights

87.40506329113923

## 更有层次感且更易修改, 但是更加复杂了的代码

- 使用一种更加有层次感, 但是更易修改的代码, 从不同类的设计的角度出发, 对其数据的获取进行specify

In [9]:
from collections import namedtuple

Grade = namedtuple('Grade', ('score', 'weight'))

class Subject:
    def __init__(self):
        self._grades = []

    def report_grade(self, score, weight):

        self._grades.append(Grade(score, weight))

    def average_grade(self):
        total, total_weight = 0, 0

        for grade in self._grades:
            total+=grade.score*grade.weight

            total_weight+=grade.weight

        return total / total_weight

In [10]:
class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject)

    def get_subject(self, name):
        return self._subjects[name]

    def average_grade(self):
        total, count = 0, 0

        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1

        return total/count

        

In [11]:
class Gradebook:
    def __init__(self):
        self._students = defaultdict(Student)

    def get_student(self, name):

        return self._students[name]

In [12]:
book = Gradebook()
albert = book.get_student('Albert Einstein')
math = albert.get_subject('Math')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)
gym = albert.get_subject('Gym')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)
print(albert.average_grade())


80.25
